In [4]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import torchvision
from torchvision import datasets, models, transforms, io
import matplotlib.pyplot as plt
import time
import os
import copy
import random
from PIL import Image
import glob
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.9.0+cu102
Torchvision Version:  0.10.0+cu102


In [5]:
def set_seed(seed=1):
    """
    Sets all random seeds.
    :param seed: int
        Seed value.
    :return: None
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [6]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))
torch.cuda.empty_cache()
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "G:/tesina/Licencias/MIcroExpressions_Data"

# Number of classes in the dataset
num_classes = 3

# Batch size for training (change depending on how much memory you have)
batch_size = 16

# Number of epochs to train for
num_epochs = 10

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

set_seed(10)


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [7]:
class MEDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir, transform=None, csv_file="", phase = ''):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.microExpresions = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.microExpresions)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        images_path = os.path.join(self.root_dir, self.phase, str(self.microExpresions.iloc[idx, 0]),
                                   str(self.microExpresions.iloc[idx, 1]), str(self.microExpresions.iloc[idx, 2]))
        array = []
        count = 0;
        for filename in glob.glob(images_path+'/*.jpg'):
            im = Image.open(filename)
            im = im.resize([224,224])
            array.append(np.asarray(im))
            count+=1;
            if count >9:
                break

        if self.microExpresions.iloc[idx, 0] == "happiness":
            emotion = 0
        elif self.microExpresions.iloc[idx, 0] == "disgust":
            emotion = 1
        else:
            emotion = 2
        sample = {'images': array, 'emotion': emotion}

        sample = self.transform(sample)
        # sample['images'] = torch.tensor(sample['images'])
        # sample['emotion'] = torch.tensor(sample['emotion'])


        return sample

In [8]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        images, emotion = sample['images'], sample['emotion']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        return {'images': torch.tensor(images),
                'emotion': torch.tensor(emotion)}

In [9]:
class MakeGray(object):
    def __call__(self, sample):
        images, emotion = sample['images'], sample['emotion']

        for im in images:
            im = transforms.Grayscale(im)

        return {'images': images, 'emotion': emotion}

In [10]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        #for phase in ['train', 'val']:
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            #for inputs, labels in dataloaders[phase]:
            for inputs in dataloaders[phase]:
                inputs['images'] = inputs['images'].to(device).float()
                inputs['emotion'] = inputs['emotion'].to(device).long()
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    inputs['images'] = inputs['images'].permute(0,4,1,2,3)
                    inputs['images'].size()

                    outputs = model(inputs['images'])
                    print(outputs)
                    print(inputs['emotion'])
                    loss = criterion(outputs, inputs['emotion'])

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs['images'].size(0)
                running_corrects += torch.sum(preds == inputs['emotion'])

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            #print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
               best_acc = epoch_acc
               best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
               val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [11]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [12]:
class MicroExpressionRecognition3D(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv3d(3, 16, kernel_size = (5,5,5), stride=(1,1,1),padding = (1,1,1),bias=True),
            nn.ReLU(),
            nn.MaxPool3d([2,2,2]),
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(788544, 128),
            nn.Dropout(0.3),
            nn.Linear(128, 3),
            nn.Softmax()
        )

    def forward(self, xb):
        return self.network(xb)

    def __repr__(self):
        return f"{self.network}"

    def __str__(self):
        summary(self.network, (1, 48, 48))


In [13]:
#model_ft = MicroExpressionRecognition()
model_ft = MicroExpressionRecognition3D()
input_size = 224

In [14]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        # transforms.Resize([input_size,input_size]),
        # transforms.RandomHorizontalFlip(),
        # transforms.RandomGrayscale(p=0.1),
        # transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.05),]), p=0.3),
        # transforms.RandomPerspective(distortion_scale=0.1, p=0.5),
        # transforms.ToTensor(),
        # transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.03, 1), value=0, inplace=False)
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ToTensor(),
        MakeGray()
    ]),
    'val': transforms.Compose([
        #transforms.Resize([input_size,input_size]),
        #transforms.RandomGrayscale(p=0.1),
        #transforms.ToTensor()
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ToTensor()
    ])
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: MEDataset(root_dir='G:\\tesina\\Licencias\\MicroExpressions_Data2',
                           transform=data_transforms[x],
                           csv_file=x+'_data.csv', phase=x) for x in ['train', 'val']}
#print(image_datasets['train'][1])

# image_datasets = MEDataset(root_dir='G:\\tesina\\Licencias\\MicroExpressions_Data2',
#                            transform=transforms.Compose([transforms.ToTensor()]),
#                            csv_file='train_data.csv', phase='train')
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=x=='train', num_workers=0) for x in ['train', 'val']}
#dataloader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, shuffle=False, num_workers=0)
#dataloader_iter = iter(dataloader)
# for step in range(len(dataloader)):
#     data= next(dataloader_iter)
#     print(f"Feature batch shape: {data['images'][0].size()}")
#     print(f"Labels batch shape: {data['emotion'][0]}")
#     new_im = data['images'][0][0]
#     print(new_im)
#     label = data['emotion']
#     plt.imshow(new_im, cmap="gray")
#     plt.show()
#train_features, train_labels = next(iter(dataloader))
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Initializing Datasets and Dataloaders...
cuda:0


In [15]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001, weight_decay = 0.01)

Params to learn:
	 network.0.weight
	 network.0.bias
	 network.5.weight
	 network.5.bias
	 network.7.weight
	 network.7.bias


In [16]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/9
----------


c:\tesina\pruebas_1\venv\lib\site-packages\torch\nn\modules\container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[1.0768e-22, 3.9828e-07, 1.0000e+00],
        [5.9049e-09, 1.7438e-10, 1.0000e+00],
        [1.6568e-08, 8.7456e-19, 1.0000e+00],
        [1.1668e-11, 1.0000e+00, 2.4613e-11],
        [5.1075e-09, 2.1757e-03, 9.9782e-01],
        [1.2251e-11, 1.0000e+00, 3.3501e-06],
        [1.5816e-07, 1.5338e-10, 1.0000e+00],
        [3.5890e-21, 2.1425e-30, 1.0000e+00],
        [5.6027e-10, 1.3120e-17, 1.0000e+00],
        [9.7395e-21, 1.0000e+00, 8.1544e-17],
        [7.4673e-13, 1.0000e+00, 3.3630e-18],
        [1.7474e-11, 3.2988e-19, 1.0000e+00],
        [2.1448e-01, 1.8426e-16, 7.8552e-01],
        [1.0370e-14, 9.9136e-01, 8.6403e-03],
        [2.6967e-02, 9.7303e-01, 6.9862e-10],
        [1.5780e-03, 9.9842e-01, 5.9468e-12]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
tensor([0, 1, 1, 0, 1, 1, 0, 0, 1, 2, 0, 1, 1, 0, 1, 2], device='cuda:0')
tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
      

In [17]:
#!jupyter nbconvert  pruebas_1.ipynb --to html